In [1]:
import time

notebook_start_time = time.time()

# Set up environment

In [2]:
import sys
from pathlib import Path


def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False


def clone_repository() -> None:
    !git clone https://github.com/decodingml/hands-on-recommender-system.git
    %cd hands-on-recommender-system/


def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()

    root_dir = str(Path().absolute())
    print("⛳️ Google Colab environment")
else:
    root_dir = str(Path().absolute().parent)
    print("⛳️ Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

⛳️ Local environment
Adding the following directory to the PYTHONPATH: /Users/pauliusztin/Documents/01_projects/hopsworks_recsys/hands-on-recommender-system


# Inference pipeline: Deploying and testing the inference pipeline 

In this notebook, we will dig into the inference pipeline and deploy it to Hopsworks as a real-time service.

## 📝 Imports

In [3]:
import warnings

warnings.filterwarnings("ignore")

from loguru import logger

from recsys import hopsworks_integration

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [4]:
project, fs = hopsworks_integration.get_feature_store()

2024-12-23 19:22:04.717 | INFO     | recsys.hopsworks_integration.feature_store:get_feature_store:13 - Loging to Hopsworks using HOPSWORKS_API_KEY env var.


2024-12-23 19:22:04,719 INFO: Initializing external client
2024-12-23 19:22:04,719 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-23 19:22:06,316 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1192098


# Deploying the ranking inference pipeline


You start by deploying your ranking model. Since it is a CatBoost model you need to implement a `Predict` class that tells Hopsworks how to load the model and how to use it:

In [5]:
ranking_deployment = hopsworks_integration.ranking_serving.HopsworksRankingModel.deploy(
    project=project
)

Uploading: 100.000%|██████████| 4491/4491 elapsed<00:01 remaining<00:00
Uploading: 100.000%|██████████| 1113/1113 elapsed<00:01 remaining<00:00


Deployment with the same name already exists. Getting existing deployment...
To create a new deployment choose a different name.


Now, we have to explicitly start the deployment:

In [6]:
ranking_deployment.start()

Deployment is already running


In [7]:
# Check logs in case of failure
# ranking_deployment.get_logs(component="transformer", tail=200)

## <span style="color:#ff5f27"> Test the ranking inference pipeline</span>


In [8]:
def get_top_recommendations(ranked_candidates, k=3):
    return [candidate[-1] for candidate in ranked_candidates["ranking"][:k]]

Let's define a dummy test example to test our ranking deployment (only the `customer_id` has to match):

In [9]:
test_ranking_input = [
        {
            "customer_id": "d327d0ad9e30085a436933dfbb7f77cf42e38447993a078ed35d93e3fd350ecf",
            "month_sin": 1.2246467991473532e-16,
            "query_emb": [
                0.214135289,
                0.571055949,
                0.330709577,
                -0.225899458,
                -0.308674961,
                -0.0115124583,
                0.0730511621,
                -0.495835781,
                0.625569344,
                -0.0438038409,
                0.263472944,
                -0.58485353,
                -0.307070434,
                0.0414443575,
                -0.321789205,
                0.966559,
            ],
            "month_cos": -1.0,
        }
    ]

# Test ranking deployment
ranked_candidates = ranking_deployment.predict(inputs=test_ranking_input)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates["predictions"], k=3)
recommendations

['803603001', '825719002', '571436009']

Check logs in case of failure:

In [10]:
# ranking_deployment.get_logs(component="transformer", tail=200)

# Deploying the query inference pipeline

In [11]:
query_model_deployment = (
    hopsworks_integration.two_tower_serving.HopsworksQueryModel.deploy(ranking_model_type="ranking")
)

2024-12-23 19:22:52,724 INFO: Closing external client and cleaning up certificates.
Connection closed.
2024-12-23 19:22:52,727 INFO: Initializing external client
2024-12-23 19:22:52,727 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-23 19:22:54,018 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1192098
2024-12-23 19:22:55,292 INFO: Closing external client and cleaning up certificates.
2024-12-23 19:22:55,296 INFO: Initializing external client
2024-12-23 19:22:55,296 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-23 19:22:56,351 INFO: Closing external client and cleaning up certificates.
Connection closed.
2024-12-23 19:22:56,356 INFO: Initializing external client
2024-12-23 19:22:56,356 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-23 19:22:57,669 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1192098
Secret created successfully, explore it at htt

Uploading: 100.000%|██████████| 2948/2948 elapsed<00:01 remaining<00:00


Deployment with the same name already exists. Getting existing deployment...
To create a new deployment choose a different name.


At this point, you have registered your deployment. To start it up you need to run:

In [12]:
query_model_deployment.start()

Deployment is already running


In [13]:
# Check logs in case of failure
# query_model_deployment.get_logs(component="transformer", tail=20)

## <span style="color:#ff5f27"> Testing the inference pipeline </span>

Define a test input example:

In [14]:
data = [
    {
        "customer_id": "d327d0ad9e30085a436933dfbb7f77cf42e38447993a078ed35d93e3fd350ecf",
        "transaction_date": "2022-11-15T12:16:25.330916",
    }
]

Test out the deployment:

In [15]:
ranked_candidates = query_model_deployment.predict(inputs=data)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates["predictions"], k=3)
recommendations

['759469001', '724772002', '446224002']

Check logs in case of failure:

In [16]:
# query_model_deployment.get_logs(component="transformer", tail=200)

# <span style="color:#ff5f27"> Stopping the Hopsworks deployments </span>

Stop the deployment when you're not using it.

In [17]:
ranking_deployment.stop()
query_model_deployment.stop()

Deployment is stopped: 100%|██████████| 4/4 [00:10<00:00,  2.66s/it]        


## <span style="color:#ff5f27"> Inspecting the deployments in Hopsworks UI </span>

View results in [Hopsworks Serverless](https://rebrand.ly/serverless-github): **Data Science → Deployments**

---

In [18]:
notebook_end_time = time.time()
notebook_execution_time = notebook_end_time - notebook_start_time

logger.info(
    f"⌛️ Notebook Execution time: {notebook_execution_time:.2f} seconds ~ {notebook_execution_time / 60:.2f} minutes"
)

2024-12-23 19:23:54.143 | INFO     | __main__:<module>:4 - ⌛️ Notebook Execution time: 113.56 seconds ~ 1.89 minutes


# <span style="color:#ff5f27">→ Next Steps </span>

The last step is to schedule the materialization jobs.